#Directory, Libraries and Data

In [1]:
# Mount Google Drive to access files in Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Path to folder
%cd /content/drive/MyDrive/Business Analyst course/Segmentation/RFM

In [3]:
# Import libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [4]:
# Load data
# E-commerce sales data
data = pd.read_csv("online_shoppers.csv",
                   encoding = "ISO-8859-1").dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


#Data Preparation

In [5]:
# Create a new column 'sales' by multiplying 'Quantity' with 'UnitPrice'
data['sales'] = data['Quantity'] * data['UnitPrice']
data.head(1)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.3


In [6]:
# Transform the 'InvoiceDate' column to datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Get the most recent date in the 'InvoiceDate' column and add one day to define the snapshot date
snapshot_date = data['InvoiceDate'].max() + timedelta(days = 1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [7]:
# Aggregate data at the customer level

df = data.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Calculate the number of days since the last purchase
    'InvoiceNo': 'count',  # Count the number of invoices per customer
    'sales': 'sum'  # Sum the total sales per customer
})

# Display the first few rows of the aggregated DataFrame
df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [8]:
# Create a new column 'monetary' representing the average sales per invoice
df['monetary'] = df.sales / df.InvoiceNo
df.head(2)

,InvoiceDate,InvoiceNo,sales,monetary
CustomerID,,,,
12346.0,326,2,0.0,0.000000
12347.0,2,182,4310.0,23.681319


In [9]:
# Drop the 'Sales' Column
df = df.drop(columns = "sales")

# See the remaining columns
df.head(0)

,InvoiceDate,InvoiceNo,monetary
CustomerID,,,


In [10]:
# Rename the columns for better clarity
df.rename(columns={
    'InvoiceDate': 'Recency',  # Days since the last purchase
    'InvoiceNo': 'Frequency'   # Number of invoices
}, inplace=True)

# Preview
df.head(1)

,Recency,Frequency,monetary
CustomerID,,,
12346.0,326,2,0.0


#RFM Preparation

In [11]:
# Create frequency groups by quantile-based binning for 'Frequency' with 4 quantiles
df['F'] = pd.qcut(x=df['Frequency'], q=4, labels=range(1, 5))

# Create monetary value groups by quantile-based binning for 'monetary' with 4 quantiles
df['M'] = pd.qcut(x=df['monetary'], q=4, labels=range(1, 5))

# Create recency groups by quantile-based binning for 'Recency' with 4 quantiles (reverse order: 4 is most recent)
df['R'] = pd.qcut(x=df['Recency'], q=4, labels=range(4, 0, -1))

# Display the first 5 rows
df.head(5)


,Recency,Frequency,monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2
12349.0,19,73,24.076027,3,4,3
12350.0,310,17,19.670588,1,3,1


In [12]:
# Calculate the RFM score by summing the values of Recency (R), Frequency (F), and Monetary (M) for each customer
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head(2)

,Recency,Frequency,monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12


In [13]:
# Create a function to segment customers based on their RFM score
def rfm_segment(df):
  if df['RFM'] >= 11:
    return 'SuperStar'                          # High RFM score: top-performing customers
  elif ((df['RFM'] >=8) and (df['RFM'] < 11)):
    return 'Future Champion'                    # Above average RFM score: potential for future high performance
  elif ((df['RFM'] >=6) and (df['RFM'] < 8)):
    return 'High Potential'                     # Average RFM score: potential for growth
  else:
    return 'Low Relevance'                      # Low RFM score: lower engagement or relevance

#RFM

In [14]:
# Apply the RFM segmentation function to the DataFrame
df['RFM_level'] = df.apply(rfm_segment, axis = 1) # Add a new column with the RFM segment label
df.tail()

,Recency,Frequency,monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
18280.0,278,10,18.060000,1,3,1,5,Low Relevance
18281.0,181,7,11.545714,1,2,1,4,Low Relevance
18282.0,8,13,13.584615,1,2,4,7,High Potential
18283.0,4,756,2.771005,4,1,4,9,Future Champion
18287.0,43,70,26.246857,3,4,3,10,Future Champion


In [15]:
# Analyze the characteristics of each RFM segment
df.groupby('RFM_level').agg({
    'Recency': 'mean',               # Average recency for each segment
    'Frequency': 'mean',             # Average frequency of purchases for each segment
    'monetary': ['mean', 'count']    # Average monetary value and count of customers in each segment
})

Recency   Frequency   monetary      
                       mean        mean       mean count
RFM_level                                               
Future Champion   32.007919  135.835973  30.169670  1768
High Potential   118.708869   39.213947  38.033024  1477
Low Relevance    221.483400   22.811421  -0.586430   753
SuperStar          9.973262  244.852941  45.483949   374